In [61]:
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import time
import sys
import os

import seaborn as sns
import gensim 

import django
import platform

if platform.node() == "srv-mcc-apsis":
    sys.path.append('/home/leey/tmv/BasicBrowser/')
else:
    # local paths
    sys.path.append('/Documents/Data/tmv/BasicBrowser/')

os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
from tmv_app.models import *
from utils.tm_mgmt import update_topic_scores
from utils.text import *
from django.db.models import Q, Count, Func, F, Sum, Value, Case, When, IntegerField

In [2]:
run_id = 1040

dts = DocTopic.objects.filter(run_id=run_id)

# total score of all DocTopics in run 1041 
dts.aggregate(
    n = Sum('score')
)

{'n': 5541434.19997849}

In [3]:
# total score of DocTopics, sorted by topic in run 1041, 
dts.values('topic').annotate(
    n = Sum('score')
)

<QuerySet [{'topic': 133048, 'n': 95752.3737478164}, {'topic': 133049, 'n': 54756.0540348548}, {'topic': 133050, 'n': 211403.523922185}, {'topic': 133051, 'n': 60982.8527237079}, {'topic': 133052, 'n': 102861.838064162}, {'topic': 133053, 'n': 154281.547869584}, {'topic': 133054, 'n': 53463.8276297085}, {'topic': 133055, 'n': 69908.9855563507}, {'topic': 133056, 'n': 306860.17252113}, {'topic': 133057, 'n': 201171.24716959}, {'topic': 133058, 'n': 92136.4556441535}, {'topic': 133059, 'n': 104507.372917855}, {'topic': 133060, 'n': 54615.0025600184}, {'topic': 133061, 'n': 1321422.09672952}, {'topic': 133062, 'n': 203062.968823175}, {'topic': 133063, 'n': 172889.836816227}, {'topic': 133064, 'n': 142649.454868041}, {'topic': 133065, 'n': 69919.1494519089}, {'topic': 133066, 'n': 190536.094146483}, {'topic': 133067, 'n': 41610.7992327013}, '...(remaining elements truncated)...']>

In [44]:
dts_list = list(dts.values('ut__document__parlperiod__n').annotate(
    n = Sum('score')
))

df_dts_list = pd.DataFrame(dts_list)

In [45]:
df_dts_list.columns = ['DocTopic_score', 'parlperiod']
df_dts_list.set_index('parlperiod')

,DocTopic_score
parlperiod,
1,485258.0
2,399660.5
3,236316.5
4,208181.9
5,425223.2
6,164192.8
7,349895.4
8,535199.9
9,187399.3


In [46]:
# DocTopic scores for topic containing 'umweltschutz' and 'steinkohl' in run 1040
# sorted by parliamentary period 

topic_period_scores = dts.values('topic__title','ut__document__parlperiod__n').annotate(n = Sum('score'))

pp_sum = dts.values('ut__document__parlperiod__n').annotate(n = Sum('score'))

df = pd.DataFrame.from_dict(list(topic_period_scores.filter(topic__title__contains='umweltschutz')))
    
df.columns = ['doctopic_score', 'topic', 'parlperiod']

df_pps = pd.DataFrame.from_dict(list(pp_sum))
df_pps.columns = ['pp_sum', 'parlperiod']

df = pd.concat([df, df_pps['pp_sum']], axis=1, join_axes=[df.index])

df['share'] = df['doctopic_score'] / df['pp_sum']

In [84]:
df

,doctopic_score,topic,parlperiod,pp_sum,share
0,92.358834,"{umweltschutz, umweltpolit, okolog}",1,485258.0,0.000190
1,214.143704,"{umweltschutz, umweltpolit, okolog}",2,399660.5,0.000536
2,157.555554,"{umweltschutz, umweltpolit, okolog}",3,236316.5,0.000667
3,287.813212,"{umweltschutz, umweltpolit, okolog}",4,208181.9,0.001383
4,315.277704,"{umweltschutz, umweltpolit, okolog}",5,425223.2,0.000741
5,1759.885582,"{umweltschutz, umweltpolit, okolog}",6,164192.8,0.010718
6,546.647735,"{umweltschutz, umweltpolit, okolog}",7,349895.4,0.001562
7,4015.918972,"{umweltschutz, umweltpolit, okolog}",8,535199.9,0.007504
8,662.131580,"{umweltschutz, umweltpolit, okolog}",9,187399.3,0.003533
9,24080.366743,"{umweltschutz, umweltpolit, okolog}",10,362873.9,0.066360


In [90]:
topic1 = Topic.objects.filter(run_id=run_id,
            title__contains='steinkohl')[0]

In [93]:
topic1.primary_dtopic.title

AttributeError: 'ManyRelatedManager' object has no attribute 'title'

In [82]:
DynamicTopic.objects.filter(run_id=RunStats.objects.get(run_id=1040))

<QuerySet []>

In [83]:
DynamicTopic.objects.filter()[0]

<DynamicTopic: {product, produc, increas}>

In [68]:
party_totals = pm.Utterance.objects.filter(
            doctopic__topic__run_id=run_id,
            speaker__party__name__isnull=False,
            speaker__party__colour__isnull=False
        ).order_by().values('speaker__party__name').annotate(
            topic_score=Sum(
                Case(
                    When(doctopic__topic=topic1,
                         then=F('doctopic__score')),
                    default=0,
                    output_field=models.FloatField()
                )
            ),
            total_score=Sum('doctopic__score'),
        ).annotate(
            topic_proportion=F('topic_score') / F('total_score')
        ).values(
            'topic_proportion',
            'speaker__party__id',
            'speaker__party__name',
            'speaker__party__colour'
        ).order_by('-topic_proportion')

---

In [55]:
df2 = pd.DataFrame.from_dict(list(topic_period_scores.filter(topic__title__contains='steinkohl')))
    
df2.columns = ['doctopic_score', 'topic', 'parlperiod']

df2 = pd.concat([df2, df_pps['pp_sum']], axis=1, join_axes=[df2.index])

df2['share'] = df2['doctopic_score'] / df2['pp_sum']

In [56]:
df2

,doctopic_score,topic,parlperiod,pp_sum,share
0,10675.920564,"{steinkohl, kohl, bergbau}",1,485258.0,0.022001
1,11378.084995,"{steinkohl, kohl, bergbau}",2,399660.5,0.028469
2,20072.015566,"{steinkohl, kohl, bergbau}",3,236316.5,0.084937
3,22035.391015,"{steinkohl, kohl, bergbau}",4,208181.9,0.105847
4,24368.074272,"{steinkohl, kohl, bergbau}",5,425223.2,0.057307
5,2557.013355,"{steinkohl, kohl, bergbau}",6,164192.8,0.015573
6,18906.160905,"{steinkohl, kohl, bergbau}",7,349895.4,0.054034
7,18416.759333,"{steinkohl, kohl, bergbau}",8,535199.9,0.034411
8,3285.589680,"{steinkohl, kohl, bergbau}",9,187399.3,0.017533
9,6431.684670,"{steinkohl, kohl, bergbau}",10,362873.9,0.017724
